# Seq2Seq
實現機器翻譯(Machine Translation)，輸入一句英文句子，輸出一句法文句子
<br>輸入 : his least liked fruit is the apple , but your least liked is the strawberry .
<br>輸出 : son fruit est moins aimé la pomme , mais votre moins aimé est la fraise .

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import time
import copy
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# 讀取數據

In [2]:
# English source data
f_eng = open('data/small_vocab_en' , 'r' , encoding = 'utf-8')
source_letter = []
source_sentence = []
while True:
    raw = f_eng.readline()
    if raw == '' : break

    sentence = raw.split('\n')[0] 
    temp_sentence = []
    for word in sentence.split(' '):
        if len(word) != 0:
            source_letter.append(word.lower())
            temp_sentence.append(word.lower())
    source_sentence.append(temp_sentence)    

    
# French target data
f_fre = open('data/small_vocab_fr', 'r', encoding='utf-8')       
target_letter = []
target_sentence = []
while True:
    raw = f_fre.readline()
    if raw == '' : break

    sentence = raw.split('\n')[0]   
    temp_sentence = []
    for word in sentence.split(' '):
        if len(word) != 0:
            target_letter.append(word.lower())
            temp_sentence.append(word.lower())
    target_sentence.append(temp_sentence)    

# 數據預處理

In [3]:
special_words = ['<PAD>' , '<UNK>' , '<GO>' , '<EOS>']

# 建造英文詞庫
source_letter = list(set(source_letter)) + special_words[:2] # 加入 '<PAD>' , '<UNK>'              
source_letter_to_int = {word : idx for idx , word in enumerate(source_letter)}   
source_int_to_letter = {idx : word for idx , word in enumerate(source_letter)}   

# 建造法文詞庫
target_letter = list(set(target_letter)) + special_words # 加入 '<PAD>' , '<UNK>' , '<GO>' , '<EOS>'       
target_letter_to_int = {word : idx for idx , word in enumerate(target_letter)}   
target_int_to_letter = {idx : word for idx , word in enumerate(target_letter)}

In [4]:
# 將所有英文詞彙轉換成index
source_int = []
for sentence in source_sentence:
    temp = []
    for letter in sentence:
        if letter in source_letter_to_int.keys():
            temp.append(source_letter_to_int[letter])  
        else:
            temp.append(source_letter_to_int['<UNK>'])
    source_int.append(temp)           

# 將所有英文詞彙轉換成index
target_int = []
for sentence in target_sentence:
    temp = []
    for letter in sentence:
        if letter in target_letter_to_int.keys():
            temp.append(target_letter_to_int[letter])
        else:
            temp.append(target_letter_to_int['<UNK>'])
    temp.append(target_letter_to_int['<EOS>'])    
    target_int.append(temp)   

In [5]:
# 超參數
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 130
# RNN Size
rnn_hidden_unit = 128
# Number of Layers
num_layers = 1
# Embedding Size
encoding_embedding_size = 100
decoding_embedding_size = 100
# Learning Rate
learning_rate = 0.001

# Build Model

## 輸入層

In [6]:
input_data = tf.placeholder(tf.int32 , [None , None] , name = 'inputs')
targets = tf.placeholder(tf.int32 , [None , None] , name = 'targets')
lr = tf.placeholder(tf.float32 , name = 'learning_rate')

source_sequence_length = tf.placeholder(tf.int32 , [None ,] , name = 'source_sequence_length')
target_sequence_length = tf.placeholder(tf.int32 , [None ,] , name = 'target_sequence_length')
# 決定target序列最大長度（之後target_sequence_length和source_sequence_length會作為feed_dict的參數）
max_target_sequence_length = tf.reduce_max(target_sequence_length , name = 'max_target_len')

## Encoder

需要對source數據進行embedding，再傳入Decoder中的RNN

In [7]:
# input_data: 輸入tensor
# rnn_hidden_unit: rnn隱層結點數量
# num_layers: rnn cell的層數
# source_sequence_length: source數據的序列長度
# source_vocab_size: source數據的詞庫大小
# encoding_embedding_size: embedding的向量維度

# Encoder embedding
'''
encoder_embed_input = tf.contrib.layers.embed_sequence(input_data , source_vocab_size , encoding_embedding_size) 
                                                  ⇕ 相當於
encoder_embeddings = tf.Variable(tf.random_uniform([source_vocab_size , encoding_embedding_size]))
encoder_embed_input = tf.nn.embedding_lookup(encoder_embeddings , input_data)

若懶得寫兩行程式可以直接用tf.contrib.layers.embed_sequence這個函數
介紹 : https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence
'''
source_vocab_size = len(source_letter_to_int)
encoder_embeddings = tf.Variable(tf.random_uniform([source_vocab_size , encoding_embedding_size]))
encoder_embed_input = tf.nn.embedding_lookup(encoder_embeddings , input_data)

def get_lstm_cell(rnn_hidden_unit):
    lstm_cell = tf.contrib.rnn.LSTMCell(rnn_hidden_unit, 
                                        initializer = tf.random_uniform_initializer(-0.1 , 0.1))
    return lstm_cell

cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_hidden_unit) for _ in range(num_layers)])

encoder_output, encoder_state = tf.nn.dynamic_rnn(cell, 
                                                  encoder_embed_input, 
                                                  sequence_length = source_sequence_length,
                                                  dtype = tf.float32)

## Decoder

In [8]:
# 預處理後的decoder輸入
# 在batch中每一筆data最前面加上<GO>，並移除最後一個字，所以每一筆data的詞的數目並無改變

# cut掉最後一個字
# ending = tf.strided_slice(targets , [0, 0] , [batch_size, -1] , [1, 1]) # 等同於 ending = tf.identity(targets[: , 0:-1])
ending = tf.identity(targets[: , 0:-1])
decoder_input = tf.concat([tf.fill([batch_size, 1] , target_letter_to_int['<GO>']) , ending] , axis = 1)

### tf.contrib.seq2seq.TrainingHelper:(Training 階段，還有其他種類的Helper)
### 訓練時採用teacher forcing，永遠把ground truth輸入給模型，不管模型前一步預測結果是否正確
此函數為Decoder端用來訓練的參數，這個函數不會把t-1時刻的輸出當作t時刻的輸入，而是把target中的真實質直接輸入給RNN<br>
主要參數是inputs與sequence_length，返回helper對象，可以做為Basic Decoder函數的參數
<br><br><br>

### tf.contrib.seq2seq.GreedyEmbeddingHelper:(Inference 階段，還有不同sample手段的Helper)
### 它和TrainingHelper的區別在於它會把t-1時刻的輸出經過embedding層作為t時刻的輸入
• greedy decoding：每一次把模型認為機率最大的 token 輸入給下一時刻<br>
• beam search decoding：每次保留 top k 的預測結果，解碼得到（近似） k best 序列 <br>
• sample decoding：每一步從模型預測的機率分布中隨機取樣一個 token 輸入給下一時刻

In [9]:
# decoding_embedding_size: embedding的向量維度
# num_layers: rnn cell的層數
# rnn_size: RNN單元的隱層結點數量
# target_sequence_length: target數據序列長度
# max_target_sequence_length: target數據序列最大長度
# encoder_state: encoder端編碼的狀態向量
# decoder_input: decoder端輸入

# 1. Embedding，需要對target數據進行embedding，再傳入Decoder中的RNN
target_vocab_size = len(target_letter_to_int)
decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size , decoding_embedding_size]))
decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings , decoder_input)

# 2. 建造Decoder中的RNN單元
def get_decoder_cell(rnn_hidden_unit):
    decoder_cell = tf.contrib.rnn.LSTMCell(rnn_hidden_unit,
                                           initializer = tf.random_uniform_initializer(-0.1 , 0.1))
    return decoder_cell
cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_hidden_unit) for _ in range(num_layers)])
 
# 3. Output全連接層
output_layer = Dense(target_vocab_size,
                     kernel_initializer = tf.truncated_normal_initializer(mean = 0.0 , stddev = 0.1))

# 4. Training decoder
with tf.variable_scope('decoder'):
    # tf.contrib.seq2seq.TrainingHelper即是採用Teacher Forcing的方法
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs = decoder_embed_input,
                                                        sequence_length = target_sequence_length,
                                                        time_major = False)
    # 建造decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 
    
    # decoder_output包含 rnn_output 與 sample_id
    # rnn_output: [batch_size, decoder_targets_length, vocab_size]，保存decode每個時刻每個單詞的概率，可以用來計算loss
    # sample_id: [batch_size], tf.int32，保存最終的編碼結果。可以表示最後的答案
    training_decoder_output ,\
    training_final_state ,\
    training_final_sequence_lengths =\
    tf.contrib.seq2seq.dynamic_decode(training_decoder,                                          
                                      impute_finished = True,
                                      maximum_iterations = max_target_sequence_length)
    
# 5. Predicting decoder 與 Training decoder共享參數
with tf.variable_scope('decoder'):
    
    tf.get_variable_scope().reuse_variables() 
    
    # 創建一個常量tensor並覆制為batch_size的大小
    start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']], dtype=tf.int32),
                           [batch_size], 
                           name = 'start_tokens')
    
    # GreedyEmbeddingHelper採取argmax抽樣演算法來得到輸出id，並且經過embedding層作為下一時刻的輸入
    predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                                 start_tokens,
                                                                 target_letter_to_int['<EOS>'])
    

    predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                         predicting_helper,
                                                         encoder_state,
                                                         output_layer)
    
    predicting_decoder_output ,\
    predicting_final_state ,\
    predicting_final_sequence_lengths =\
    tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                      impute_finished = True,
                                      maximum_iterations = max_target_sequence_length)                    

In [10]:
training_logits = tf.identity(training_decoder_output.rnn_output ,  name = 'logits')
predicting_logits = tf.identity(predicting_decoder_output.sample_id ,  name = 'predictions')

'''
target_sequence_length : [4 , 2 , 3]

max_target_sequence_length : 8

=> masks的輸出長這樣 : 1 1 1 1 0 0 0 0  (4)
                       1 1 0 0 0 0 0 0  (2)
                       1 1 1 0 0 0 0 0  (3)
-> 0的部分代表是補0的地方，不列入loss的計算，可以加快運算速度
'''                              
masks = tf.sequence_mask(target_sequence_length , 
                         max_target_sequence_length , 
                         dtype = tf.float32, 
                         name = 'masks')


with tf.variable_scope('optimization'):        
    # Loss function
    cost = tf.contrib.seq2seq.sequence_loss(training_logits,
                                            targets,
                                            masks)

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1. , 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Batches

In [11]:
def get_batches(source , target , index = None , on_train = False):  
    if on_train:
        source = np.array(source)[index]    
        target = np.array(target)[index]   
    else:
        source = np.array(source)
        target = np.array(target)
        
    # 決定source與target中的最大長度
    source_max_length , target_max_length = 0 , 0  
    for vob_source , vob_target in zip(source , target):
        if len(vob_source) > source_max_length:
            source_max_length = len(vob_source)    
        if len(vob_target) > target_max_length:
            target_max_length = len(vob_target)  
 
    # 分別對source與target補source_letter_to_int['<PAD>']與target_letter_to_int['<PAD>']到最大長度  
    source_pad , target_pad = [] , []
    source_len , target_len = [] , []
    for source_sentence , target_sentence in zip(source , target):
        source_len.append(len(source_sentence)) # 收集source中每個sencentence的長度
        temp_source = source_sentence.copy()
        while len(temp_source) < source_max_length:
            temp_source.append(source_letter_to_int['<PAD>']) 
        source_pad.append(temp_source)
        
        target_len.append(len(target_sentence)) # 收集target中每個sencentence的長度
        temp_target = target_sentence.copy()
        while len(temp_target) < target_max_length:
            temp_target.append(target_letter_to_int['<PAD>']) 
        target_pad.append(temp_target) 
        
    return np.array(source_pad) , np.array(target_pad) , np.array(source_len) , np.array(target_len)         

In [12]:
# 將數據集分割為train和validation
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]
# 留出一個batch進行驗證
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]

## Training

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch_i in range(0 , 20):
    
    # 在每進行一個epoch前，把每個batch的index先決定出來
    batch_index = []
    temp = []
    count = 0 # 隨機決定index的開頭 
    while len(batch_index) <= 1059:  # 1個batch裡只有1059筆資料
        temp.append(count)
        count += 1
        if len(temp) == batch_size:
            batch_index.append(temp)
            temp = []
        if count == len(train_source):
            count = 0

    for batch_i in range(0 , 1059):
        train_source_batch_pad , \
        train_target_batch_pad , \
        train_source_batch_length , \
        train_target_batch_length = get_batches(source = train_source ,
                                                target = train_target , 
                                                index = batch_index[batch_i] , 
                                                on_train = True)
        
        _ , loss , predicting_logits_result = \
        sess.run([train_op , cost , predicting_logits],
                 feed_dict = {input_data : train_source_batch_pad ,
                              targets : train_target_batch_pad ,
                              source_sequence_length : train_source_batch_length ,
                              target_sequence_length : train_target_batch_length  ,
                              lr: learning_rate})


        if batch_i % 50 == 0: # 每隔50個輪查看一下結果
            valid_source_pad , \
            valid_target_pad , \
            valid_source_length , \
            valid_target_length = get_batches(source = valid_source ,
                                              target = valid_target)
            
            validation_loss = sess.run(cost, 
                                       feed_dict = {input_data : valid_source_pad ,
                                                    targets : valid_target_pad ,
                                                    source_sequence_length : valid_source_length , 
                                                    target_sequence_length : valid_target_length})
            
            print('Epoch : {}/{} \nBatch : {}/{} \nTraining Loss : {:.3f} \nValidation loss: {:.3f}'
                  .format(epoch_i , epochs , 
                          batch_i , len(train_source) // batch_size ,
                          loss , validation_loss))

            index = np.random.randint(batch_size) # 隨機決定一筆data，查看翻譯的結果

            source_visualization = []
            for i in train_source_batch_pad[index]:
                if source_int_to_letter[i] == '<PAD>': break
                source_visualization.append(source_int_to_letter[i])
            source_visualization = ' '.join(source_visualization)

            target_visualization = []
            for i in train_target_batch_pad[index]:
                if target_int_to_letter[i] == '<EOS>': break
                target_visualization.append(target_int_to_letter[i])
            target_visualization = ' '.join(target_visualization)

            predict_visualization = []
            for i in predicting_logits_result[index]:
                if target_int_to_letter[i] == '<EOS>': break
                predict_visualization.append(target_int_to_letter[i])
            predict_visualization = ' '.join(predict_visualization)

            print('Source : {}'.format(source_visualization))
            print('Target : {}'.format(target_visualization))
            print('Predict : {}\n'.format(predict_visualization))
    
    
# 保存模型
saver = tf.train.Saver()
saver.save(sess , 'trained_model/save_net')
print('Model Trained and Saved')

Epoch : 0/60 
Batch : 0/1059 
Training Loss : 5.894 
Validation loss: 5.743
Source : you dislike oranges , apples , and grapefruit .
Target : vous n'aimez pas les oranges , les pommes et le pamplemousse .
Predict : singes ours pamplemousses pamplemousses grand de de grosses grosses grosses grosses grosses grosses grosses grosses grosses grosses grosses grosses grosses grosses

Epoch : 0/60 
Batch : 50/1059 
Training Loss : 3.520 
Validation loss: 3.418
Source : she likes grapefruit , mangoes , and bananas .
Target : elle aime le pamplemousse , les mangues et les bananes .
Predict : les est les les , , et les .

Epoch : 0/60 
Batch : 100/1059 
Training Loss : 2.668 
Validation loss: 2.682
Source : new jersey is usually snowy during october , and it is usually beautiful in june .
Target : new jersey est généralement enneigée en octobre , et il est généralement beau en juin .
Predict : la est est parfois en en , mais il est est en en .

Epoch : 0/60 
Batch : 150/1059 
Training Loss : 2.17

Epoch : 1/60 
Batch : 200/1059 
Training Loss : 0.446 
Validation loss: 0.443
Source : china is sometimes nice during june , and it is rainy in winter .
Target : chine est parfois agréable en juin , et il pleut en hiver .
Predict : chine est parfois agréable en juin , et il est agréable en hiver .

Epoch : 1/60 
Batch : 250/1059 
Training Loss : 0.436 
Validation loss: 0.418
Source : the united states is sometimes wonderful during may , but it is sometimes wet in august .
Target : les états-unis est parfois merveilleux au mois de mai , mais il est parfois humide en août .
Predict : les états-unis est parfois merveilleux au mois de mai , mais il est parfois humide en août .

Epoch : 1/60 
Batch : 300/1059 
Training Loss : 0.392 
Validation loss: 0.388
Source : paris is never freezing during may , and it is never wet in november .
Target : paris est jamais le gel en mai , et il est jamais humide en novembre .
Predict : paris est jamais le gel au mois de mai , et il est jamais merveilleux

Epoch : 2/60 
Batch : 450/1059 
Training Loss : 0.118 
Validation loss: 0.123
Source : paris is usually beautiful during winter , and it is usually relaxing in autumn .
Target : paris est généralement beau pendant l' hiver , et il est relaxant habituellement à l' automne .
Predict : paris est généralement beau pendant l' hiver , et il est généralement agréable en août .

Epoch : 2/60 
Batch : 500/1059 
Training Loss : 0.108 
Validation loss: 0.106
Source : india is never hot during fall , but it is usually pleasant in may .
Target : l' inde est jamais chaud à l'automne , mais il est généralement agréable en mai .
Predict : l' inde est jamais chaud pendant l' automne , mais il est généralement agréable en mai .

Epoch : 2/60 
Batch : 550/1059 
Training Loss : 0.105 
Validation loss: 0.104
Source : they dislike oranges , limes , and pears .
Target : ils n'aiment pas les oranges , citrons verts , et les poires .
Predict : ils n'aiment pas les oranges , les citrons verts et les poires .

E

Epoch : 3/60 
Batch : 600/1059 
Training Loss : 0.064 
Validation loss: 0.062
Source : new jersey is usually nice during august , and it is wonderful in december .
Target : new jersey est généralement agréable au mois d' août , et il est merveilleux en décembre .
Predict : new jersey est généralement agréable au mois d' août , et il est merveilleux en décembre .

Epoch : 3/60 
Batch : 650/1059 
Training Loss : 0.056 
Validation loss: 0.062
Source : he dislikes bananas , limes , and apples .
Target : il n'aime les bananes , citrons verts , et les pommes .
Predict : il n'aime les bananes , citrons verts , et les pommes .

Epoch : 3/60 
Batch : 700/1059 
Training Loss : 0.076 
Validation loss: 0.055
Source : california is never dry during summer , but it is never chilly in winter .
Target : californie est jamais à sec pendant l' été , mais jamais froid en hiver .
Predict : californie est jamais à sec pendant l' été , mais il est jamais froid en hiver .

Epoch : 3/60 
Batch : 750/1059 
Tra

Epoch : 4/60 
Batch : 800/1059 
Training Loss : 0.048 
Validation loss: 0.039
Source : the united states is never cold during march , and it is never pleasant in september .
Target : les états-unis ne fait jamais froid en mars , et il est jamais agréable en septembre .
Predict : les états-unis ne fait jamais froid en mars , et il est jamais agréable en septembre .

Epoch : 4/60 
Batch : 850/1059 
Training Loss : 0.048 
Validation loss: 0.039
Source : the orange is their least liked fruit , but the banana is your least liked .
Target : l'orange est leur fruit moins aimé , mais la banane est votre moins aimé .
Predict : l'orange est leur fruit moins aimé , mais la banane est votre moins aimé .

Epoch : 4/60 
Batch : 900/1059 
Training Loss : 0.079 
Validation loss: 0.038
Source : china is sometimes rainy during june , and it is busy in may .
Target : chine est parfois pluvieux en juin , et il est occupé en mai .
Predict : chine est parfois pluvieux en juin , et il est occupé en mai .

Ep

...

Epoch : 14/60 
Batch : 600/1059 
Training Loss : 0.015 
Validation loss: 0.017
Source : france is warm during march , but it is never mild in december .
Target : france est chaud au mois de mars , mais il est jamais doux en décembre .
Predict : france est chaud au mois de mars , mais il est jamais doux en décembre .

Epoch : 14/60 
Batch : 650/1059 
Training Loss : 0.012 
Validation loss: 0.015
Source : china is sometimes freezing during april , but it is sometimes quiet in september .
Target : la chine est parfois le gel en avril , mais il est parfois calme en septembre .
Predict : la chine est parfois le gel en avril , mais il est parfois calme en septembre .

Epoch : 14/60 
Batch : 700/1059 
Training Loss : 0.017 
Validation loss: 0.014
Source : her least favorite fruit is the apple , but my least favorite is the peach .
Target : son fruit préféré moins est la pomme , mais mon préféré moins est la pêche .
Predict : son fruit préféré moins est la pomme , mais mon préféré moins est la

Epoch : 15/60 
Batch : 800/1059 
Training Loss : 0.010 
Validation loss: 0.020
Source : he likes mangoes , grapefruit , and pears .
Target : il aime la mangue , le pamplemousse et les poires .
Predict : il aime la mangue , le pamplemousse et les poires .

Epoch : 15/60 
Batch : 850/1059 
Training Loss : 0.011 
Validation loss: 0.019
Source : the united states is rainy during april , and it is quiet in august .
Target : les états-unis est pluvieux en avril , et il est calme en août .
Predict : les états-unis est pluvieux en avril , et il est calme en août .

Epoch : 15/60 
Batch : 900/1059 
Training Loss : 0.026 
Validation loss: 0.019
Source : she likes peaches , lemons , and grapes .
Target : elle aime les pêches , les citrons et les raisins .
Predict : elle aime les pêches , les citrons et les raisins .

Epoch : 15/60 
Batch : 950/1059 
Training Loss : 0.012 
Validation loss: 0.015
Source : her least liked fruit is the apple , but our least liked is the mango .
Target : son fruit est

Epoch : 17/60 
Batch : 0/1059 
Training Loss : 0.009 
Validation loss: 0.014
Source : california is never hot during fall , but it is usually relaxing in april .
Target : californie est jamais chaud à l'automne , mais il est relaxant habituellement en avril .
Predict : californie est jamais chaud à l'automne , mais il est relaxant habituellement en avril .

Epoch : 17/60 
Batch : 50/1059 
Training Loss : 0.010 
Validation loss: 0.018
Source : the united states is usually wet during april , but it is sometimes rainy in october .
Target : les états-unis est généralement humide en avril , mais il est parfois pluvieux en octobre .
Predict : les états-unis est généralement humide en avril , mais il est parfois pluvieux en octobre .

Epoch : 17/60 
Batch : 100/1059 
Training Loss : 0.007 
Validation loss: 0.012
Source : france is never relaxing during summer , and it is never dry in november .
Target : la france ne se détendre pendant l' été , et il est jamais sec en novembre .
Predict : la 

Epoch : 18/60 
Batch : 250/1059 
Training Loss : 0.017 
Validation loss: 0.011
Source : new jersey is freezing during december , and it is never cold in january .
Target : new jersey est le gel en décembre , et il ne fait jamais froid en janvier .
Predict : new jersey est le gel en décembre , et il ne fait jamais froid en janvier .

Epoch : 18/60 
Batch : 300/1059 
Training Loss : 0.009 
Validation loss: 0.017
Source : he saw the rusty yellow automobile .
Target : il a vu la voiture rouillée jaune .
Predict : il a vu la voiture rouillée jaune .

Epoch : 18/60 
Batch : 350/1059 
Training Loss : 0.007 
Validation loss: 0.014
Source : california is sometimes snowy during january , but it is sometimes rainy in april .
Target : californie est parfois enneigée en janvier , mais il est parfois pluvieux en avril .
Predict : californie est parfois enneigée en janvier , mais il est parfois pluvieux en avril .

Epoch : 18/60 
Batch : 400/1059 
Training Loss : 0.007 
Validation loss: 0.014
Source 

Epoch : 19/60 
Batch : 450/1059 
Training Loss : 0.006 
Validation loss: 0.014
Source : my favorite fruit is the lime , but her favorite is the lemon .
Target : mon fruit préféré est la chaux , mais son préféré est le citron .
Predict : mon fruit préféré est la chaux , mais son préféré est le citron .

Epoch : 19/60 
Batch : 500/1059 
Training Loss : 0.005 
Validation loss: 0.014
Source : china is never wet during august , but it is never wonderful in december .
Target : chine est jamais humide au mois d' août , mais il est jamais merveilleux en décembre .
Predict : chine est jamais humide au mois d' août , mais il est jamais merveilleux en décembre .

Epoch : 19/60 
Batch : 550/1059 
Training Loss : 0.011 
Validation loss: 0.014
Source : new jersey is usually wet during autumn , but it is usually chilly in june .
Target : new jersey est généralement humide en automne , mais il est généralement froid en juin .
Predict : new jersey est généralement humide en automne , mais il est généra

## Testing

In [20]:
import os    
sess = tf.Session()
new_saver = tf.train.import_meta_graph(os.path.join('trained_model/save_net.meta'))
new_saver.restore(sess, tf.train.latest_checkpoint(os.path.join('trained_model')))

graph = tf.get_default_graph()
input_data = graph.get_tensor_by_name('inputs:0')
targets = graph.get_tensor_by_name('targets:0')
source_sequence_length = graph.get_tensor_by_name('source_sequence_length:0')
target_sequence_length = graph.get_tensor_by_name('target_sequence_length:0')
logits = graph.get_tensor_by_name('predictions:0')

input_sentence = 'i dislike grapefruit , lemons , and peaches .'
test_source = []
for letter in input_sentence.split(' '):
    if letter in source_letter_to_int.keys():
        test_source.append(source_letter_to_int[letter])
    elif letter not in source_letter_to_int.values():
        test_source.append(source_letter_to_int['<UNK>'])
test_source = [test_source] * batch_size
test_source_length = [len(i) for i in test_source]
           
test_target = [i for i in range(0 , 100)] # test_target輸入的值可以隨便選，只要長度大於test_source即可
test_target = [test_target] * batch_size
test_target_length = [len(i) for i in test_target]

test_source = np.array(test_source)
test_target = np.array(test_target)
test_source_length = np.array(test_source_length)
test_target_length = np.array(test_target_length)

answer = sess.run(logits, feed_dict = {input_data : test_source ,
                                       targets : test_target ,
                                       source_sequence_length : test_source_length ,
                                       target_sequence_length : test_target_length})

answer = answer[0 , :]
answer_to_letter = []
for num in answer:
    answer_to_letter.append(target_int_to_letter[num])

print('輸入的英文句子 : {}'.format(input_sentence))  
print('google翻譯的法文句子 : {}'.format("je n'aime pamplemousses , les citrons et les pêches ."))  
print('model翻譯的法文句子 : {}'.format(' '.join(answer_to_letter)))      

輸入的英文句子 : i dislike grapefruit , lemons , and peaches .
google翻譯的法文句子 : je n'aime pamplemousses , les citrons et les pêches .
model翻譯的法文句子 : je n'aime pamplemousses , les citrons et les pêches . <EOS>
